# Neural Image Style Transfer Lab
One of the really cool applications of deep learning that has come lately is the idea of neural image style transfers.  Take a look at [https://deepart.io](https://deepart.io) to get an idea of what it can do. Based off [the paper on Neural image style transfer](https://arxiv.org/pdf/1508.06576.pdf), we can take an image (source) and a style image (like a famous painting) and generate an image based on the style.  To begin, let's import all the required libraries. 

In [25]:
import scipy.misc
from nist import *
import numpy as np
import tensorflow as tf
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 # make it so we can accept bigger images. 

Now we need to select which images we will use.  This is the part where you can change the images to process.  We can do that later.  For now, lets use the images that are in the lab.  First we have a ```content_image_name```.  This is the image that we want modified in a cool new style.  We also have the ```style_image_name```.  This is the Van Gogh image of _Starry Night_.  We will use the style from this painting and apply it to the ```content_image```.  To add your own images for use, simply upload them into the [images directory](./images).  Don't worry, these images will go away when the sandbox terminates. 

In [26]:
content_image_name = "images/Devnet_Sandbox_CLus2_300DPI.png"  # This is the image we want to enhance with a style
style_image_name = "images/starry_night@2000x1500.jpg" # This is the style image.  We will take this image to use.

<table><tr><td><img src="images/Devnet_Sandbox_CLus2_300DPI.png" width=400 alt="DEVI picture"/></td>
    <td><img src="images/starry_night@2000x1500.jpg" width=400 alt="Starry Night" /></td></tr></table>

Our program expects the images to be 2000px width x 1500px height so we will need to reformat the images to make it 2000x1500.  Then we need to take the image and turn it into an matrix.  The matrix will then be a 2000x1500x3 matrix.  The 3 comes from the RGB spectrum of colors. Let's create a function that does that.  

It turns out that a lot of time spent on data science involves data wrangling and manipulating data to get it into the right form. 

In [48]:
def format_to_size(image):
    """
    Takes in an image, converts the image to a 2000x1500 size then turns it into an array
    of dimension IMAGE_WIDTHxIMAGE_HEIGHTx3
    """
    img = Image.open(image).convert("RGB") #open with RGB
    img = img.resize((CONFIG.IMAGE_WIDTH,CONFIG.IMAGE_HEIGHT), Image.ANTIALIAS) # Resize to IMAGE_WIDTHxIMAGE_HEIGHT
    img = reshape_and_normalize_image(np.asarray(img)) # Normalize with mean values
    return img

Now let's run our images through the function and check the array sizes that we get from it. 

In [52]:
content_image = format_to_size(content_image_name)
style_image = format_to_size(style_image_name)
print(content_image.shape)
print(style_image.shape)

(1, 900, 1200, 3)
(1, 900, 1200, 3)


We will now run these two images through the neural network.  The neural network itself, while interesting, is beyond the scope of this lab to understand.  What is important to understand is that many of the algorithms that have been created for deep neural networks are open source and available to use, even if you don't understand too many of the details behind them.  In this case a model has already been created and trained.  The [paper](https://arxiv.org/pdf/1508.06576.pdf) uses the imagenet-vgg-verydeep-19.mat model.  We use the exact same one and run our algorithm through it. Even with the GPU it takes about 5 minutes for this next part to run.  So sit back as it runs through the neural network 100 times and each time enhances the style. You can run this code on your laptop as well and it takes about 45 minutes to run through the whole thing.  

In [55]:
# reset the Tensorfloat graph and get the variables ready.
tf.reset_default_graph()
with tf.Session() as sess:
    generated_image = generate_noise_image(content_image)  # add some noise to give some texture.
    model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat") # load the model. 
    # Assign the content image to be the input of the VGG model.
    sess.run(model['input'].assign(content_image))
    out = model['conv4_2']
    a_C = sess.run(out)
    a_G = out
    # Compute the content cost
    J_content = compute_content_cost(a_C, a_G)
    # Assign the input of the model to be the "style" image
    sess.run(model['input'].assign(style_image))
    # Compute the style cost
    J_style = compute_style_cost(sess, model, STYLE_LAYERS)
    J = total_cost(J_content, J_style, alpha=10, beta=40)
    # define optimizer (1 line)
    optimizer = tf.train.AdamOptimizer(2.0)
    # define train_step (1 line)
    train_step = optimizer.minimize(J)
    model_nn(sess, model, train_step, J, J_content, J_style, generated_image)

Iteration 0 :
total cost = 3553763000.0
content cost = 17302.645
style cost = 88839750.0
Iteration 20 :
total cost = 369577820.0
content cost = 23543.45
style cost = 9233560.0
Iteration 40 :
total cost = 135825820.0
content cost = 24721.4
style cost = 3389465.2
Iteration 60 :
total cost = 78894220.0
content cost = 25163.18
style cost = 1966064.9
Iteration 80 :
total cost = 57181860.0
content cost = 25389.006
style cost = 1423199.2


<img src="output/generated_image.jpg" width=400 alt="DEVI picture"/>

You can check in the [outputs folder](./output/) of this lab for the images that were created.  A new image is written every 20 iterations.  Contrast and find the ones you like the best. You may also decide to upload new pictures to the images directory and modify the code to generate your own art.  You can change the style and the content!  